# 第１回　データの読み込みと最短経路探索の改良

## データ読み込みと整形

In [32]:
import os
import numpy as np
import networkx as nx
import csv
import time
import pandas as pd
import copy

### まずはデータ読み込み
**pandasを使ってみる**

In [3]:
path = '../datafile/SiouxFalls_node.tntp'
df = pd.read_csv(path, '\t', index_col=0)# 指定pathのファイルを，タブ区切りで，始めの列を見出しにして読む
df

,X,Y,;
Node,,,
1,-96.770420,43.612828,;
2,-96.711251,43.605813,;
3,-96.774303,43.572962,;
4,-96.747168,43.563654,;
5,-96.731569,43.564034,;
6,-96.711644,43.587586,;
7,-96.693423,43.563844,;
8,-96.711382,43.562324,;
9,-96.731241,43.548596,;


In [24]:
len(df)

24

In [6]:
netfile = '../datafile/SiouxFalls_net.tntp'
net = pd.read_csv(netfile, skiprows=8, sep='\t')# 指定pathのファイルを，始めの8行を除いて，タブ区切りで読む

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)
net

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,2,25900.200640,6,6,0.15,4,0,0,1
1,1,3,23403.473190,4,4,0.15,4,0,0,1
2,2,1,25900.200640,6,6,0.15,4,0,0,1
3,2,6,4958.180928,5,5,0.15,4,0,0,1
4,3,1,23403.473190,4,4,0.15,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...
71,23,22,5000.000000,4,4,0.15,4,0,0,1
72,23,24,5078.508436,2,2,0.15,4,0,0,1
73,24,13,5091.256152,4,4,0.15,4,0,0,1
74,24,21,4885.357564,3,3,0.15,4,0,0,1


In [31]:
len(net)

76

In [11]:
net["capacity"][3]

4958.180928

In [37]:
# ベクトルにも変換可能

tate = np.array(net["capacity"])
yoko = np.array(net[0:1])[0]

print("縦 =", tate)
print("横 =", yoko)

縦 = [25900.20064  23403.47319  25900.20064   4958.180928 23403.47319
 17110.52372  23403.47319  17110.52372  17782.7941    4908.82673
 17782.7941    4947.995469 10000.        4958.180928  4947.995469
  4898.587646  7841.81131  23403.47319   4898.587646  7841.81131
  5050.193156  5045.822583 10000.        5050.193156 13915.78842
 13915.78842  10000.       13512.00155   4854.917717  4993.510694
  4908.82673  10000.        4908.82673   4876.508287 23403.47319
  4908.82673  25900.20064  25900.20064   5091.256152  4876.508287
  5127.526119  4924.790605 13512.00155   5127.526119 14564.75315
  9599.180565  5045.822583  4854.917717  5229.910063 19679.89671
  4993.510694  5229.910063  4823.950831 23403.47319  19679.89671
 23403.47319  14564.75315   4823.950831  5002.607563 23403.47319
  5002.607563  5059.91234   5075.697193  5059.91234   5229.910063
  4885.357564  9599.180565  5075.697193  5229.910063  5000.
  4924.790605  5000.        5078.508436  5091.256152  4885.357564
  5078.508436]
横 = [1

### ネットワークのデータを必要なものだけ整形してみる

In [28]:
N = {}
L = {}

for i in range(len(net)):

    from_node = net["init_node"][i]
    to_node   = net["term_node"][i]
    ff_time   = net["free_flow_time"][i]
    
    L[i+1] = {}
    L[i+1]["index"]     = i
    L[i+1]["from_node"] = from_node
    L[i+1]["to_node"]   = to_node
    L[i+1]["ff_time"]   = ff_time
    
    if N.get(from_node):# Nの中に from_node があるなら
        N[from_node]['outlink_list'] = N[from_node]['outlink_list'] + [i+1]
    else:
        N[from_node] = {}
        N[from_node]["index"] = from_node-1
        N[from_node]['outlink_list'] = [i+1]
        N[from_node]['inlink_list']  = []
            
    if N.get(to_node):# Nの中に to_node があるなら
        N[to_node]['inlink_list'] = N[to_node]['inlink_list'] + [i+1]
    else:
        N[to_node] = {}
        N[to_node]["index"] = to_node-1
        N[to_node]['outlink_list'] = []
        N[to_node]['inlink_list']  = [i+1]

In [29]:
len(N)

24

In [33]:
N

{1: {'index': 0, 'outlink_list': [1, 2], 'inlink_list': [3, 5]},
 2: {'index': 1, 'outlink_list': [3, 4], 'inlink_list': [1, 14]},
 3: {'index': 2, 'outlink_list': [5, 6, 7], 'inlink_list': [2, 8, 35]},
 6: {'index': 5, 'outlink_list': [14, 15, 16], 'inlink_list': [4, 12, 19]},
 4: {'index': 3, 'outlink_list': [8, 9, 10], 'inlink_list': [6, 11, 31]},
 12: {'index': 11, 'outlink_list': [35, 36, 37], 'inlink_list': [7, 33, 38]},
 5: {'index': 4, 'outlink_list': [11, 12, 13], 'inlink_list': [9, 15, 23]},
 11: {'index': 10,
  'outlink_list': [31, 32, 33, 34],
  'inlink_list': [10, 27, 36, 40]},
 9: {'index': 8, 'outlink_list': [23, 24, 25], 'inlink_list': [13, 21, 26]},
 8: {'index': 7,
  'outlink_list': [19, 20, 21, 22],
  'inlink_list': [16, 17, 24, 47]},
 7: {'index': 6, 'outlink_list': [17, 18], 'inlink_list': [20, 54]},
 18: {'index': 17, 'outlink_list': [54, 55, 56], 'inlink_list': [18, 50, 60]},
 16: {'index': 15,
  'outlink_list': [47, 48, 49, 50],
  'inlink_list': [22, 29, 52, 55]

In [35]:
L

{1: {'index': 0, 'from_node': 1, 'to_node': 2, 'ff_time': 6},
 2: {'index': 1, 'from_node': 1, 'to_node': 3, 'ff_time': 4},
 3: {'index': 2, 'from_node': 2, 'to_node': 1, 'ff_time': 6},
 4: {'index': 3, 'from_node': 2, 'to_node': 6, 'ff_time': 5},
 5: {'index': 4, 'from_node': 3, 'to_node': 1, 'ff_time': 4},
 6: {'index': 5, 'from_node': 3, 'to_node': 4, 'ff_time': 4},
 7: {'index': 6, 'from_node': 3, 'to_node': 12, 'ff_time': 4},
 8: {'index': 7, 'from_node': 4, 'to_node': 3, 'ff_time': 4},
 9: {'index': 8, 'from_node': 4, 'to_node': 5, 'ff_time': 2},
 10: {'index': 9, 'from_node': 4, 'to_node': 11, 'ff_time': 6},
 11: {'index': 10, 'from_node': 5, 'to_node': 4, 'ff_time': 2},
 12: {'index': 11, 'from_node': 5, 'to_node': 6, 'ff_time': 4},
 13: {'index': 12, 'from_node': 5, 'to_node': 9, 'ff_time': 5},
 14: {'index': 13, 'from_node': 6, 'to_node': 2, 'ff_time': 5},
 15: {'index': 14, 'from_node': 6, 'to_node': 5, 'ff_time': 4},
 16: {'index': 15, 'from_node': 6, 'to_node': 8, 'ff_time

In [38]:
N[6]["outlink_list"]

[14, 15, 16]

### 同じことは，pandasを使わなくてもできる
**こっちのほうが記憶容量の節約になるかも．**

In [56]:
N = {}
L = {}

f = open("../datafile/SiouxFalls_net.tntp", "r")

flag = False
i = 0
for line in f:# ここでファイルを一行ずつ読んでいる．一気に読まないので使用メモリが少ない（多分）
    if flag == True:
        link_parameter = line.split()# 一つの文字列を，スペースとか改行とかタブとかで分割する
    
        from_node = int(link_parameter[0])
        to_node = int(link_parameter[1])
        ff_time = float(link_parameter[4])
        
        L[i+1] = {}
        L[i+1]["index"]     = i
        L[i+1]["from_node"] = from_node
        L[i+1]["to_node"]   = to_node
        L[i+1]["ff_time"]   = ff_time
        
        if N.get(from_node):# Nの中に from_node があるなら
            N[from_node]['outlink_list'] = N[from_node]['outlink_list'] + [i+1]
        else:
            N[from_node] = {}
            N[from_node]["index"] = from_node-1
            N[from_node]['outlink_list'] = [i+1]
            N[from_node]['inlink_list']  = []
            
        if N.get(to_node):# Nの中に to_node があるなら
            N[to_node]['inlink_list'] = N[to_node]['inlink_list'] + [i+1]
        else:
            N[to_node] = {}
            N[to_node]["index"] = to_node-1
            N[to_node]['outlink_list'] = []
            N[to_node]['inlink_list']  = [i+1]
            
        i = i + 1
    
    
    if line[0] == "~":
        flag = True
    
    
f.close()

In [57]:
L

{1: {'index': 0, 'from_node': 1, 'to_node': 2, 'ff_time': 6.0},
 2: {'index': 1, 'from_node': 1, 'to_node': 3, 'ff_time': 4.0},
 3: {'index': 2, 'from_node': 2, 'to_node': 1, 'ff_time': 6.0},
 4: {'index': 3, 'from_node': 2, 'to_node': 6, 'ff_time': 5.0},
 5: {'index': 4, 'from_node': 3, 'to_node': 1, 'ff_time': 4.0},
 6: {'index': 5, 'from_node': 3, 'to_node': 4, 'ff_time': 4.0},
 7: {'index': 6, 'from_node': 3, 'to_node': 12, 'ff_time': 4.0},
 8: {'index': 7, 'from_node': 4, 'to_node': 3, 'ff_time': 4.0},
 9: {'index': 8, 'from_node': 4, 'to_node': 5, 'ff_time': 2.0},
 10: {'index': 9, 'from_node': 4, 'to_node': 11, 'ff_time': 6.0},
 11: {'index': 10, 'from_node': 5, 'to_node': 4, 'ff_time': 2.0},
 12: {'index': 11, 'from_node': 5, 'to_node': 6, 'ff_time': 4.0},
 13: {'index': 12, 'from_node': 5, 'to_node': 9, 'ff_time': 5.0},
 14: {'index': 13, 'from_node': 6, 'to_node': 2, 'ff_time': 5.0},
 15: {'index': 14, 'from_node': 6, 'to_node': 5, 'ff_time': 4.0},
 16: {'index': 15, 'from_no

In [58]:
N

{1: {'index': 0, 'outlink_list': [1, 2], 'inlink_list': [3, 5]},
 2: {'index': 1, 'outlink_list': [3, 4], 'inlink_list': [1, 14]},
 3: {'index': 2, 'outlink_list': [5, 6, 7], 'inlink_list': [2, 8, 35]},
 6: {'index': 5, 'outlink_list': [14, 15, 16], 'inlink_list': [4, 12, 19]},
 4: {'index': 3, 'outlink_list': [8, 9, 10], 'inlink_list': [6, 11, 31]},
 12: {'index': 11, 'outlink_list': [35, 36, 37], 'inlink_list': [7, 33, 38]},
 5: {'index': 4, 'outlink_list': [11, 12, 13], 'inlink_list': [9, 15, 23]},
 11: {'index': 10,
  'outlink_list': [31, 32, 33, 34],
  'inlink_list': [10, 27, 36, 40]},
 9: {'index': 8, 'outlink_list': [23, 24, 25], 'inlink_list': [13, 21, 26]},
 8: {'index': 7,
  'outlink_list': [19, 20, 21, 22],
  'inlink_list': [16, 17, 24, 47]},
 7: {'index': 6, 'outlink_list': [17, 18], 'inlink_list': [20, 54]},
 18: {'index': 17, 'outlink_list': [54, 55, 56], 'inlink_list': [18, 50, 60]},
 16: {'index': 15,
  'outlink_list': [47, 48, 49, 50],
  'inlink_list': [22, 29, 52, 55]

# 課題：
## 1. Heap 構造を自分で作ってくる（class を作ってくると良い）．

- 何らかの数値属性を持った要素（「所要時間ラベルを持ったノード」など）のリストをヒープに変換できること．

- 変換してできたヒープに要素を追加，ヒープから最小要素を取得，という２操作ができること．

「ヒープとは？」については今日の説明に加えて，青本の pp.139 - 140 にも載っているので確認のこと．ググってもいろいろ出てくるはず．

## 2. 自作の Heap 構造を使ってダイクストラ法を作ってくる．

- まず紹介したN, Lを使って，heapを使わないダイクストラ法（前回のやつ）を完成させる．

- 次に，heapを使ってダイクストラ法を作る．

- 最後に，前回と今回のダイクストラ法を大きい（1000リンクくらいの）ネットワークに適用して速さを確認する．

ネットワークのデータは[ここ](https://github.com/bstabler/TransportationNetworks)から落としてくる．